In [ ]:
!pip install diffusers transformers accelerate peft safetensors ftfy pillow numpy matplotlib tqdm opencv-python scikit-image

In [ ]:
# importsa nd setup
import os
import random
import numpy as np
import cv2
import torch
import torch.nn.functional as F
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from diffusers import StableDiffusionInpaintPipeline, DDPMScheduler
from peft import LoraConfig, get_peft_model
from torch.optim import AdamW
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# Configuration
IMAGE_SIZE = 512
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Paths
DAMAGED_DIR = Path("/content/drive/MyDrive/DL_Project/damaged images")
RAW_DIR = Path("/content/drive/MyDrive/DL_Project/Raw Images")
BASE_TRAIN_PATH = "/content/drive/MyDrive/DL_Project/train"
CLEAN_DIR = Path(BASE_TRAIN_PATH) / "clean images"
MASK_DIR = Path(BASE_TRAIN_PATH) / "mask"

# Create directories
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
MASK_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# preprocessing functions
def center_crop(img: Image.Image) -> Image.Image:
    width, height = img.size
    crop_size = min(width, height)
    left = (width - crop_size) // 2
    top = (height - crop_size) // 2
    right = left + crop_size
    bottom = top + crop_size
    return img.crop((left, top, right, bottom))

def normalize_color(img: Image.Image) -> Image.Image:
    img_cv = np.array(img)
    img_yuv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    img_norm = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    return Image.fromarray(img_norm)

def remove_metadata(img: Image.Image) -> Image.Image:
    data = list(img.getdata())
    new_img = Image.new(img.mode, img.size)
    new_img.putdata(data)
    return new_img

# Process raw images to clean
if RAW_DIR.exists():
    print("Preprocessing raw images")
    for filename in tqdm(sorted(os.listdir(RAW_DIR))):
        if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
            continue
        in_path = RAW_DIR / filename
        out_name = Path(filename).stem + ".png"
        out_path = CLEAN_DIR / out_name
        try:
            img = Image.open(in_path).convert("RGB")
            img = center_crop(img)
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
            img = normalize_color(img)
            img = remove_metadata(img)
            img.save(out_path, format="PNG")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    print(f"Clean images saved to: {CLEAN_DIR}")

# Standardize damaged images
print("Standardizing damaged images")
for fname in tqdm(sorted(os.listdir(DAMAGED_DIR))):
    p = DAMAGED_DIR / fname
    try:
        img = Image.open(p).convert("RGB")
        if img.size != (IMAGE_SIZE, IMAGE_SIZE) or p.suffix.lower() != ".png":
            img = center_crop(img).resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
            img = normalize_color(img)
            img = remove_metadata(img)
            out_name = Path(p).stem + ".png"
            img.save(DAMAGED_DIR / out_name, format="PNG")
            if p.suffix.lower() != ".png":
                p.unlink(missing_ok=True)
    except Exception as e:
        print(f"Skipping {p}: {e}")

In [ ]:
def generate_optimized_damage_mask(img_pil, debug=False):
    """Optimized damage detection that preserves detected features"""
    img = np.array(img_pil)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 1. Multi-scale Laplacian for cracks
    crack_mask = np.zeros_like(gray, dtype=np.float32)
    for ksize in [3, 5, 7]:
        laplacian = cv2.Laplacian(gray, cv2.CV_64F, ksize=ksize)
        laplacian = np.abs(laplacian)
        crack_mask += cv2.normalize(laplacian, None, 0, 1, cv2.NORM_MINMAX)

    crack_mask = cv2.normalize(crack_mask, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    _, crack_bin = cv2.threshold(crack_mask, 30, 255, cv2.THRESH_BINARY)

    # 2. Sobel edges
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    sobel = cv2.normalize(sobel, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    _, sobel_mask = cv2.threshold(sobel, 40, 255, cv2.THRESH_BINARY)

    # 3. Canny edges
    edges = cv2.Canny(gray, 30, 100)

    # 4. LAB color space damage
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Bright damage (paint loss)
    _, bright_mask = cv2.threshold(l, 210, 255, cv2.THRESH_BINARY)

    # Dark damage (stains, shadows from cracks)
    _, dark_mask = cv2.threshold(l, 50, 255, cv2.THRESH_BINARY_INV)

    # 5. Texture damage detection
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    texture_diff = cv2.absdiff(gray, blur)
    _, texture_mask = cv2.threshold(texture_diff, 15, 255, cv2.THRESH_BINARY)

    # Combine all masks with equal weight
    combined = np.zeros_like(gray, dtype=np.float32)
    combined += crack_bin.astype(np.float32)
    combined += sobel_mask.astype(np.float32)
    combined += edges.astype(np.float32)
    combined += bright_mask.astype(np.float32)
    combined += dark_mask.astype(np.float32)
    combined += texture_mask.astype(np.float32)

    # Normalize to 0-255
    combined = cv2.normalize(combined, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Threshold to get binary mask
    _, combined_binary = cv2.threshold(combined, 50, 255, cv2.THRESH_BINARY)

    # MINIMAL morphological operations (preserve detail)
    kernel_tiny = np.ones((2, 2), np.uint8)
    kernel_small = np.ones((3, 3), np.uint8)

    # Remove only the smallest noise
    combined_binary = cv2.morphologyEx(combined_binary, cv2.MORPH_OPEN, kernel_tiny, iterations=1)

    # Close small gaps
    combined_binary = cv2.morphologyEx(combined_binary, cv2.MORPH_CLOSE, kernel_small, iterations=2)

    # Moderate dilation to ensure coverage
    combined_binary = cv2.dilate(combined_binary, kernel_small, iterations=2)

    # Create soft mask with progressive blurring
    mask_soft = combined_binary.astype(np.float32)
    mask_soft = cv2.GaussianBlur(mask_soft, (11, 11), 0)
    mask_soft = cv2.GaussianBlur(mask_soft, (11, 11), 0)

    # Boost intensity
    mask_soft = np.clip(mask_soft * 1.8, 0, 255).astype(np.uint8)

    # Calculate coverage
    mask_coverage = np.sum(mask_soft > 20) / mask_soft.size * 100

    if debug:
        fig, axes = plt.subplots(2, 4, figsize=(18, 9))
        axes[0, 0].imshow(crack_bin, cmap='gray')
        axes[0, 0].set_title('Laplacian')
        axes[0, 0].axis('off')

        axes[0, 1].imshow(sobel_mask, cmap='gray')
        axes[0, 1].set_title('Sobel')
        axes[0, 1].axis('off')

        axes[0, 2].imshow(edges, cmap='gray')
        axes[0, 2].set_title('Canny')
        axes[0, 2].axis('off')

        axes[0, 3].imshow(texture_mask, cmap='gray')
        axes[0, 3].set_title('Texture')
        axes[0, 3].axis('off')

        axes[1, 0].imshow(bright_mask, cmap='gray')
        axes[1, 0].set_title('Bright Damage')
        axes[1, 0].axis('off')

        axes[1, 1].imshow(dark_mask, cmap='gray')
        axes[1, 1].set_title('Dark Damage')
        axes[1, 1].axis('off')

        axes[1, 2].imshow(combined_binary, cmap='gray')
        axes[1, 2].set_title('Combined Binary')
        axes[1, 2].axis('off')

        axes[1, 3].imshow(mask_soft, cmap='gray')
        axes[1, 3].set_title(f'Final ({mask_coverage:.1f}% coverage)')
        axes[1, 3].axis('off')

        plt.tight_layout()
        plt.show()

    print(f"Mask coverage: {mask_coverage:.2f}%")

    return mask_soft

# Test on the problematic image
test_files = sorted(os.listdir(DAMAGED_DIR))
test_file = test_files[0]  # or specify the one from your screenshot

print(f"Testing on: {test_file}")
test_img = Image.open(DAMAGED_DIR / test_file).convert("RGB")

test_mask = generate_optimized_damage_mask(test_img, debug=True)

# Show results
plt.figure(figsize=(18, 5))

plt.subplot(1, 4, 1)
plt.imshow(test_img)
plt.title("Damaged Image", fontsize=14)
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(test_mask, cmap='gray')
plt.title("Generated Mask", fontsize=14)
plt.axis('off')

plt.subplot(1, 4, 3)
overlay = np.array(test_img).copy()
mask_colored = cv2.applyColorMap(test_mask, cv2.COLORMAP_JET)
overlay = cv2.addWeighted(overlay, 0.7, mask_colored, 0.3, 0)
plt.imshow(overlay)
plt.title("Mask Overlay", fontsize=14)
plt.axis('off')

plt.subplot(1, 4, 4)
# Show mask intensity histogram
plt.hist(test_mask.flatten(), bins=50, color='blue', alpha=0.7)
plt.xlabel('Intensity')
plt.ylabel('Pixel Count')
plt.title('Mask Intensity Distribution')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Mask Statistics:")
print(f"Mean intensity: {np.mean(test_mask):.2f}")
print(f"Max intensity: {np.max(test_mask)}")
print(f"Non-zero pixels: {np.count_nonzero(test_mask):,} ({np.count_nonzero(test_mask)/test_mask.size*100:.2f}%)")
print("="*60)

In [ ]:
print("Generating masks for all images...")

mask_stats = []

for fname in tqdm(sorted(os.listdir(DAMAGED_DIR))):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    damaged_path = DAMAGED_DIR / fname
    img_pil = Image.open(damaged_path).convert("RGB")

    mask_np = generate_optimized_damage_mask(img_pil, debug=False)
    mask_pil = Image.fromarray(mask_np).convert("L")

    # Track statistics
    coverage = np.sum(mask_np > 20) / mask_np.size * 100
    mean_intensity = np.mean(mask_np)
    mask_stats.append({
        'file': fname,
        'coverage': coverage,
        'mean_intensity': mean_intensity
    })

    save_path = MASK_DIR / fname
    mask_pil.save(save_path)

print("\nMask generation complete!")
print(f"\nOverall Statistics:")
print(f"Average coverage: {np.mean([s['coverage'] for s in mask_stats]):.2f}%")
print(f"Average intensity: {np.mean([s['mean_intensity'] for s in mask_stats]):.2f}")

# Show images with lowest coverage (potential problems)
mask_stats_sorted = sorted(mask_stats, key=lambda x: x['coverage'])
print(f"\nImages with lowest mask coverage (check these):")
for stat in mask_stats_sorted[:5]:
    print(f"  {stat['file']}: {stat['coverage']:.2f}% coverage")

In [ ]:
# Visually inspect several masks
num_samples = 6
sample_files = random.sample([f for f in os.listdir(DAMAGED_DIR) if f.endswith('.png')], min(num_samples, len(os.listdir(DAMAGED_DIR))))

fig, axes = plt.subplots(num_samples, 3, figsize=(12, num_samples*3))

for idx, fname in enumerate(sample_files):
    damaged = Image.open(DAMAGED_DIR / fname).convert("RGB")
    mask = Image.open(MASK_DIR / fname).convert("L")

    # Show damaged
    axes[idx, 0].imshow(damaged)
    axes[idx, 0].set_title(f"{fname}", fontsize=10)
    axes[idx, 0].axis('off')

    # Show mask
    axes[idx, 1].imshow(mask, cmap='gray')
    coverage = np.sum(np.array(mask) > 20) / np.array(mask).size * 100
    axes[idx, 1].set_title(f"Mask ({coverage:.1f}%)", fontsize=10)
    axes[idx, 1].axis('off')

    # Show overlay
    overlay = np.array(damaged).copy()
    mask_np = np.array(mask)
    mask_colored = cv2.applyColorMap(mask_np, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(overlay, 0.7, mask_colored, 0.3, 0)
    axes[idx, 2].imshow(overlay)
    axes[idx, 2].set_title("Overlay", fontsize=10)
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# setup model config

print("Loading Stable Diffusion Inpainting model...")

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    use_safetensors=False,
    variant=None
).to(DEVICE)

pipe.scheduler = DDPMScheduler.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    subfolder="scheduler"
)

# Optimize memory
pipe.safety_checker = None
pipe.requires_safety_checker = False
pipe.enable_attention_slicing()
pipe.enable_vae_slicing()

# Freeze VAE and text encoder
pipe.vae.eval()
pipe.text_encoder.eval()
pipe.vae.requires_grad_(False)
pipe.text_encoder.requires_grad_(False)

# UNet in FP32 for training stability
pipe.unet.to(device=DEVICE, dtype=torch.float32)
pipe.unet.requires_grad_(False)

print("Base model ready!")

In [ ]:
# Enhanced LoRA configuration
lora_config = LoraConfig(
    r=32,  # Increased rank for better capacity
    lora_alpha=64,  # Increased alpha for stronger adaptation
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.1,
    bias="none"
)

pipe.unet = get_peft_model(pipe.unet, lora_config)
pipe.unet.print_trainable_parameters()

print("Enhanced LoRA attached successfully!")

In [ ]:
# dataset and dataloader

class RestorationDataset(Dataset):
    def __init__(self):
        clean_files = set(os.listdir(CLEAN_DIR))
        damaged_files = set(os.listdir(DAMAGED_DIR))
        mask_files = set(os.listdir(MASK_DIR))

        self.files = sorted(clean_files & damaged_files & mask_files)
        print(f"Total aligned samples: {len(self.files)}")

        self.img_tf = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
        ])

        self.mask_tf = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]

        clean = Image.open(CLEAN_DIR / fname).convert("RGB")
        damaged = Image.open(DAMAGED_DIR / fname).convert("RGB")
        mask = Image.open(MASK_DIR / fname).convert("L")

        return {
            "clean": self.img_tf(clean),
            "damaged": self.img_tf(damaged),
            "mask": self.mask_tf(mask),
        }

dataset = RestorationDataset()

loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

print(f"DataLoader ready | Batches: {len(loader)}")

In [ ]:
pipe.unet.train()

LR = 5e-6  # Lower learning rate for stability
EPOCHS = 10  # More epochs for better learning
ACCUMULATION_STEPS = 4  # Gradient accumulation

optimizer = AdamW(pipe.unet.parameters(), lr=LR, weight_decay=1e-4)

# Learning rate scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-7)

print("Starting enhanced training...")

for epoch in range(EPOCHS):
    total_loss = 0.0
    valid_steps = 0

    progress_bar = tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for batch_idx, batch in enumerate(progress_bar):
        clean = batch["clean"].to(DEVICE).to(torch.float32)
        damaged = batch["damaged"].to(DEVICE).to(torch.float32)
        mask = torch.clamp(batch["mask"].to(DEVICE).to(torch.float32), 0, 1)

        # Encode to latents
        with torch.no_grad():
            clean_latents = pipe.vae.encode(clean.to(torch.float16)).latent_dist.sample().to(torch.float32) * 0.18215
            damaged_latents = pipe.vae.encode(damaged.to(torch.float16)).latent_dist.sample().to(torch.float32) * 0.18215

            # Resize mask to latent space
            mask_latent = F.interpolate(mask, size=clean_latents.shape[-2:], mode="nearest")
            mask_latent = (mask_latent > 0.3).to(torch.float32)  # Lower threshold for better coverage

        # Sample noise and timesteps
        noise = torch.randn_like(clean_latents)
        timesteps = torch.randint(
            0,
            pipe.scheduler.config.num_train_timesteps,
            (clean_latents.shape[0],),
            device=DEVICE
        ).long()

        # Add noise to clean latents
        noisy_latents = pipe.scheduler.add_noise(clean_latents, noise, timesteps)

        # Create masked input
        masked_latents = damaged_latents * (1 - mask_latent) + noisy_latents * mask_latent

        # Concatenate inputs
        latent_model_input = torch.cat([noisy_latents, masked_latents, mask_latent], dim=1)

        # Text conditioning
        prompts = ["professional photo restoration, repair damage, preserve details"] * clean_latents.shape[0]
        inputs = pipe.tokenizer(
            prompts,
            padding="max_length",
            truncation=True,
            max_length=pipe.tokenizer.model_max_length,
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.to(DEVICE)

        with torch.no_grad():
            encoder_hidden_states = pipe.text_encoder(input_ids)[0].to(torch.float32)

        # Forward pass
        model_pred = pipe.unet(
            latent_model_input,
            timesteps,
            encoder_hidden_states=encoder_hidden_states
        ).sample

        # Enhanced loss computation
        # Weighted MSE emphasizing damaged regions
        weight_mask = mask_latent * 3.0 + 0.5  # Higher weight on damaged areas
        mse_loss = F.mse_loss(model_pred * weight_mask, noise * weight_mask)

        # Perceptual loss component (L1 loss on predictions)
        l1_loss = F.l1_loss(model_pred, noise)

        # Combined loss
        loss = mse_loss + 0.1 * l1_loss

        # Gradient accumulation
        loss = loss / ACCUMULATION_STEPS
        loss.backward()

        if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(pipe.unet.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * ACCUMULATION_STEPS
        valid_steps += 1

        progress_bar.set_postfix({"loss": loss.item() * ACCUMULATION_STEPS})

    # Step scheduler
    scheduler.step()

    avg_loss = total_loss / max(1, valid_steps)
    print(f"Epoch {epoch+1} DONE | Avg Loss: {avg_loss:.6f} | LR: {scheduler.get_last_lr()[0]:.2e}")

# Save trained model
SAVE_PATH = "/content/drive/MyDrive/DL_Project/Final Trained Enhanced"
os.makedirs(SAVE_PATH, exist_ok=True)

pipe.unet.save_pretrained(SAVE_PATH)
print(f"Enhanced LoRA saved at: {SAVE_PATH}")

In [ ]:
print("Loading model for inference")

# Load base model
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32
).to(DEVICE)

pipe.safety_checker = None
pipe.requires_safety_checker = False

# Load trained LoRA
pipe.load_lora_weights("/content/drive/MyDrive/DL_Project/Final Trained Enhanced")

pipe.enable_attention_slicing()
pipe.enable_vae_slicing()

pipe.unet.eval()
pipe.vae.eval()
pipe.text_encoder.eval()

print("Trained LoRA loaded for testing!")

In [ ]:
# Create comparison directory
COMPARISON_DIR = Path("/content/drive/MyDrive/DL_Project/comparisons")
COMPARISON_DIR.mkdir(parents=True, exist_ok=True)

print(f"Creating comparison images")

damaged_files = sorted([f for f in os.listdir(DAMAGED_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

for fname in tqdm(damaged_files[:20], desc="Creating comparisons"):  # First 20 for quick preview
    try:
        # Load all versions
        damaged_pil = Image.open(DAMAGED_DIR / fname).convert("RGB").resize((512, 512))
        mask_pil = Image.open(MASK_DIR / fname).convert("L").resize((512, 512))
        clean_pil = Image.open(CLEAN_DIR / fname).convert("RGB").resize((512, 512))
        restored_name = Path(fname).stem + "_restored.png"
        restored_pil = Image.open(RESTORED_DIR / restored_name).convert("RGB")

        # Create comparison grid
        fig, axes = plt.subplots(1, 4, figsize=(20, 5))

        axes[0].imshow(damaged_pil)
        axes[0].set_title("Damaged", fontsize=14)
        axes[0].axis("off")

        axes[1].imshow(mask_pil, cmap="gray")
        axes[1].set_title("Mask", fontsize=14)
        axes[1].axis("off")

        axes[2].imshow(restored_pil)
        axes[2].set_title("Restored", fontsize=14)
        axes[2].axis("off")

        axes[3].imshow(clean_pil)
        axes[3].set_title("Original Clean", fontsize=14)
        axes[3].axis("off")

        plt.tight_layout()

        # Save comparison
        comparison_name = Path(fname).stem + "_comparison.png"
        plt.savefig(COMPARISON_DIR / comparison_name, dpi=150, bbox_inches='tight')
        plt.close()

    except Exception as e:
        print(f"Error creating comparison for {fname}: {e}")

print(f"\nComparison images saved to: {COMPARISON_DIR}")

In [ ]:
def hybrid_restoration(damaged_pil, mask_pil, use_ai=True):
    """
    Combines OpenCV inpainting (for style preservation)
    with light AI enhancement (for quality)
    """
    damaged_np = np.array(damaged_pil).astype(np.float32)
    mask_np = np.array(mask_pil).astype(np.float32) / 255.0

    # Check if mask has enough content
    mask_coverage = np.sum(mask_np > 0.1) / mask_np.size * 100

    if mask_coverage < 1.0:
        # Too little damage, return original
        return damaged_pil

    # Step 1: OpenCV inpainting (style-preserving base)
    damaged_uint8 = damaged_np.astype(np.uint8)
    mask_uint8 = (mask_np * 255).astype(np.uint8)
    mask_inv = 255 - mask_uint8

    inpainted_ns = cv2.inpaint(damaged_uint8, mask_inv, 7, cv2.INPAINT_NS)
    inpainted_telea = cv2.inpaint(damaged_uint8, mask_inv, 7, cv2.INPAINT_TELEA)

    # Blend both OpenCV methods
    cv_result = (0.6 * inpainted_ns + 0.4 * inpainted_telea).astype(np.uint8)

    if use_ai and mask_coverage > 5.0:
        # Step 2: Light AI refinement only if significant damage
        cv_result_pil = Image.fromarray(cv_result)

        # Reduce mask strength for AI
        mask_reduced = (mask_np * 0.5).clip(0, 1)
        mask_reduced = cv2.GaussianBlur(mask_reduced, (21, 21), 0)
        mask_reduced_pil = Image.fromarray((mask_reduced * 255).astype(np.uint8)).convert("L")

        with torch.no_grad():
            ai_result = pipe(
                prompt="subtle texture repair, match surrounding style exactly, painting texture",
                negative_prompt="photorealistic, photograph, smooth, modern, different face, identity change",
                image=cv_result_pil,
                mask_image=mask_reduced_pil,
                num_inference_steps=30,
                guidance_scale=3.5,
                strength=0.45
            ).images[0]

        ai_np = np.array(ai_result).astype(np.float32)
        cv_np = np.array(cv_result).astype(np.float32)

        # Blend AI with OpenCV (60% OpenCV, 40% AI)
        blended = cv_np * 0.6 + ai_np * 0.4
    else:
        blended = cv_result.astype(np.float32)

    # Step 3: Final blending with original
    mask_smooth = cv2.GaussianBlur(mask_np, (31, 31), 0)
    mask_smooth = np.clip(mask_smooth * 0.8, 0, 1)  # Reduce to 80%

    final = blended * mask_smooth[..., None] + damaged_np * (1 - mask_smooth[..., None])

    # Color correction to match original
    for c in range(3):
        orig_mean = np.mean(damaged_np[:, :, c])
        orig_std = np.std(damaged_np[:, :, c])
        final_mean = np.mean(final[:, :, c])
        final_std = np.std(final[:, :, c])

        if final_std > 0:
            final[:, :, c] = (final[:, :, c] - final_mean) * (orig_std / final_std) + orig_mean

    final = np.clip(final, 0, 255).astype(np.uint8)

    return Image.fromarray(final)

# Test
fname = "005.png"
print(f"Testing hybrid restoration on: {fname}")

damaged_pil = Image.open(DAMAGED_DIR / fname).convert("RGB").resize((512, 512))
mask_pil = Image.open(MASK_DIR / fname).convert("L").resize((512, 512))
clean_pil = Image.open(CLEAN_DIR / fname).convert("RGB").resize((512, 512))

# Try both modes
restored_cv_only = hybrid_restoration(damaged_pil, mask_pil, use_ai=False)
restored_hybrid = hybrid_restoration(damaged_pil, mask_pil, use_ai=True)

# Metrics
clean_np = np.array(clean_pil).astype(np.float32)
damaged_np = np.array(damaged_pil).astype(np.float32)
cv_np = np.array(restored_cv_only).astype(np.float32)
hybrid_np = np.array(restored_hybrid).astype(np.float32)

psnr_d = psnr(clean_np, damaged_np, data_range=255)
psnr_cv = psnr(clean_np, cv_np, data_range=255)
psnr_h = psnr(clean_np, hybrid_np, data_range=255)

ssim_d = ssim(clean_np, damaged_np, data_range=255, channel_axis=2)
ssim_cv = ssim(clean_np, cv_np, data_range=255, channel_axis=2)
ssim_h = ssim(clean_np, hybrid_np, data_range=255, channel_axis=2)

# Display
fig, axes = plt.subplots(1, 5, figsize=(22, 5))

axes[0].imshow(damaged_pil)
axes[0].set_title(f"Damaged\nPSNR: {psnr_d:.2f}", fontsize=11)
axes[0].axis('off')

axes[1].imshow(mask_pil, cmap='gray')
axes[1].set_title("Mask", fontsize=11)
axes[1].axis('off')

axes[2].imshow(restored_cv_only)
axes[2].set_title(f"OpenCV Only\nPSNR: {psnr_cv:.2f} ({psnr_cv-psnr_d:+.2f})", fontsize=11)
axes[2].axis('off')

axes[3].imshow(restored_hybrid)
axes[3].set_title(f"Hybrid\nPSNR: {psnr_h:.2f} ({psnr_h-psnr_d:+.2f})", fontsize=11)
axes[3].axis('off')

axes[4].imshow(clean_pil)
axes[4].set_title("Clean Target", fontsize=11)
axes[4].axis('off')

plt.tight_layout()
plt.show()

print(f"\nOpenCV Only: PSNR {psnr_cv-psnr_d:+.2f} dB, SSIM {ssim_cv-ssim_d:+.4f}")
print(f"Hybrid:      PSNR {psnr_h-psnr_d:+.2f} dB, SSIM {ssim_h-ssim_d:+.4f}")

In [ ]:
RESTORED_DIR = Path("/content/drive/MyDrive/DL_Project/restored images hybrid")
RESTORED_DIR.mkdir(parents=True, exist_ok=True)

print("Generating hybrid restorations...")

use_ai = True  # Set to False for OpenCV-only (faster)

for fname in tqdm(sorted(os.listdir(DAMAGED_DIR))):
    if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        damaged_pil = Image.open(DAMAGED_DIR / fname).convert("RGB").resize((512, 512))
        mask_pil = Image.open(MASK_DIR / fname).convert("L").resize((512, 512))

        restored = hybrid_restoration(damaged_pil, mask_pil, use_ai=use_ai)

        output_name = Path(fname).stem + "_restored.png"
        restored.save(RESTORED_DIR / output_name)

    except Exception as e:
        print(f"\nError: {fname}: {e}")

print(f"\nSaved to: {RESTORED_DIR}")

In [ ]:
import csv

# Create metrics file
METRICS_PATH = Path("/content/drive/MyDrive/DL_Project/restoration_metrics.csv")

print("Computing and saving metrics for all restored images...")

# First, check what files exist
print("\nChecking directories...")
print(f"Damaged images: {len(list(DAMAGED_DIR.glob('*.png')))} files")
print(f"Clean images: {len(list(CLEAN_DIR.glob('*.png')))} files")
print(f"Restored images: {len(list(RESTORED_DIR.glob('*.png')))} files")

# Get list of restored files (they have _restored suffix)
restored_files = list(RESTORED_DIR.glob("*_restored.png"))
print(f"\nFound {len(restored_files)} restored images")

if len(restored_files) == 0:
    print("\nERROR: No restored images found!")
    print("Please run the restoration cell (Cell 9A) first to generate restored images.")
    print(f"Expected location: {RESTORED_DIR}")
else:
    results = []

    for restored_path in tqdm(restored_files, desc="Computing metrics"):
        try:
            # Extract original filename (remove _restored suffix)
            original_fname = restored_path.stem.replace("_restored", "") + ".png"

            # Check if corresponding files exist
            clean_path = CLEAN_DIR / original_fname
            damaged_path = DAMAGED_DIR / original_fname

            if not clean_path.exists():
                print(f"Warning: Clean image not found for {original_fname}")
                continue

            if not damaged_path.exists():
                print(f"Warning: Damaged image not found for {original_fname}")
                continue

            # Load images
            clean_pil = Image.open(clean_path).convert("RGB").resize((512, 512))
            damaged_pil = Image.open(damaged_path).convert("RGB").resize((512, 512))
            restored_pil = Image.open(restored_path).convert("RGB").resize((512, 512))

            # Convert to numpy
            clean_np = np.array(clean_pil).astype(np.float32)
            damaged_np = np.array(damaged_pil).astype(np.float32)
            restored_np = np.array(restored_pil).astype(np.float32)

            # Compute metrics
            psnr_damaged = psnr(clean_np, damaged_np, data_range=255)
            ssim_damaged = ssim(clean_np, damaged_np, data_range=255, channel_axis=2)
            psnr_restored = psnr(clean_np, restored_np, data_range=255)
            ssim_restored = ssim(clean_np, restored_np, data_range=255, channel_axis=2)

            results.append({
                'filename': original_fname,
                'psnr_damaged': psnr_damaged,
                'ssim_damaged': ssim_damaged,
                'psnr_restored': psnr_restored,
                'ssim_restored': ssim_restored,
                'psnr_improvement': psnr_restored - psnr_damaged,
                'ssim_improvement': ssim_restored - ssim_damaged
            })

        except Exception as e:
            print(f"Error computing metrics for {restored_path.name}: {e}")

    if len(results) > 0:
        # Save to CSV
        with open(METRICS_PATH, 'w', newline='') as csvfile:
            fieldnames = ['filename', 'psnr_damaged', 'ssim_damaged', 'psnr_restored',
                         'ssim_restored', 'psnr_improvement', 'ssim_improvement']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            writer.writerows(results)

        print(f"\nMetrics saved to: {METRICS_PATH}")

        # Print summary statistics
        psnr_improvements = [r['psnr_improvement'] for r in results]
        ssim_improvements = [r['ssim_improvement'] for r in results]

        print("\n" + "="*60)
        print("SUMMARY STATISTICS")
        print("="*60)
        print(f"Total images processed: {len(results)}")

        # Damaged vs Clean
        avg_psnr_damaged = np.mean([r['psnr_damaged'] for r in results])
        avg_ssim_damaged = np.mean([r['ssim_damaged'] for r in results])

        # Restored vs Clean
        avg_psnr_restored = np.mean([r['psnr_restored'] for r in results])
        avg_ssim_restored = np.mean([r['ssim_restored'] for r in results])

        print(f"\nDamaged  → Clean | PSNR: {avg_psnr_damaged:.2f} ± {np.std([r['psnr_damaged'] for r in results]):.2f} dB | SSIM: {avg_ssim_damaged:.4f} ± {np.std([r['ssim_damaged'] for r in results]):.4f}")
        print(f"Restored → Clean | PSNR: {avg_psnr_restored:.2f} ± {np.std([r['psnr_restored'] for r in results]):.2f} dB | SSIM: {avg_ssim_restored:.4f} ± {np.std([r['ssim_restored'] for r in results]):.4f}")

        print(f"\nIMPROVEMENT:")
        print(f"PSNR Gain: {np.mean(psnr_improvements):+.2f} ± {np.std(psnr_improvements):.2f} dB")
        print(f"SSIM Gain: {np.mean(ssim_improvements):+.4f} ± {np.std(ssim_improvements):.4f}")

        if len(psnr_improvements) > 0:
            print(f"\nBest PSNR improvement: {max(psnr_improvements):+.2f} dB ({results[psnr_improvements.index(max(psnr_improvements))]['filename']})")
            print(f"Worst PSNR improvement: {min(psnr_improvements):+.2f} dB ({results[psnr_improvements.index(min(psnr_improvements))]['filename']})")
            print(f"\nBest SSIM improvement: {max(ssim_improvements):+.4f} ({results[ssim_improvements.index(max(ssim_improvements))]['filename']})")
            print(f"Worst SSIM improvement: {min(ssim_improvements):+.4f} ({results[ssim_improvements.index(min(ssim_improvements))]['filename']})")

        print("="*60)

        # Create visualization of improvements
        plt.figure(figsize=(14, 5))

        plt.subplot(1, 2, 1)
        plt.scatter(range(len(psnr_improvements)), psnr_improvements, alpha=0.6, s=50)
        plt.axhline(y=0, color='r', linestyle='--', linewidth=1, label='No improvement')
        plt.xlabel('Image Index')
        plt.ylabel('PSNR Improvement (dB)')
        plt.title('PSNR Improvement per Image')
        plt.grid(alpha=0.3)
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.scatter(range(len(ssim_improvements)), ssim_improvements, alpha=0.6, s=50, color='green')
        plt.axhline(y=0, color='r', linestyle='--', linewidth=1, label='No improvement')
        plt.xlabel('Image Index')
        plt.ylabel('SSIM Improvement')
        plt.title('SSIM Improvement per Image')
        plt.grid(alpha=0.3)
        plt.legend()

        plt.tight_layout()
        plt.show()

    else:
        print("\nERROR: No results were computed. Check if files exist and match.")

In [ ]:
# Select a random test image to test restoration
fname = random.choice(os.listdir(DAMAGED_DIR))
print(f"Testing on: {fname}")

damaged_pil = Image.open(DAMAGED_DIR / fname).convert("RGB").resize((512, 512))
mask_pil = Image.open(MASK_DIR / fname).convert("L").resize((512, 512))
clean_pil = Image.open(CLEAN_DIR / fname).convert("RGB").resize((512, 512))

# Perform restoration
with torch.no_grad():
    restored_pil = pipe(
        prompt="professional museum-quality photo restoration, repair all damage, preserve original details and colors, natural skin texture, sharp details",
        negative_prompt="blur, artifacts, distortion, oversaturated, fake, synthetic, unnatural, scratches, cracks",
        image=damaged_pil,
        mask_image=mask_pil,
        num_inference_steps=75,
        guidance_scale=7.5,
        strength=0.95
    ).images[0]

# Display results
plt.figure(figsize=(20, 5))

plt.subplot(1, 4, 1)
plt.imshow(damaged_pil)
plt.title("Damaged", fontsize=14)
plt.axis("off")

plt.subplot(1, 4, 2)
plt.imshow(mask_pil, cmap="gray")
plt.title("Mask", fontsize=14)
plt.axis("off")

plt.subplot(1, 4, 3)
plt.imshow(restored_pil)
plt.title("Restored (Enhanced LoRA)", fontsize=14)
plt.axis("off")

plt.subplot(1, 4, 4)
plt.imshow(clean_pil)
plt.title("Original Clean", fontsize=14)
plt.axis("off")

plt.tight_layout()
plt.show()

# Compute metrics
p_damaged, s_damaged = psnr(np.array(clean_pil), np.array(damaged_pil), data_range=255), \
                       ssim(np.array(clean_pil), np.array(damaged_pil), data_range=255, channel_axis=2)
p_restored, s_restored = psnr(np.array(clean_pil), np.array(restored_pil), data_range=255), \
                         ssim(np.array(clean_pil), np.array(restored_pil), data_range=255, channel_axis=2)

print(f"\nDamaged  → Clean | PSNR: {p_damaged:.2f} dB | SSIM: {s_damaged:.4f}")
print(f"Restored → Clean | PSNR: {p_restored:.2f} dB | SSIM: {s_restored:.4f}")
print(f"Improvement      | PSNR: {p_restored - p_damaged:+.2f} dB | SSIM: {s_restored - s_damaged:+.4f}")

In [ ]:
NUM_SAMPLES = 50

psnr_damaged_list = []
ssim_damaged_list = []
psnr_restored_list = []
ssim_restored_list = []

def tensor_to_pil(t, is_mask=False):
    """Convert tensor to PIL image"""
    if t.min() < 0:
        t = (t + 1) / 2

    t = t.squeeze().cpu().numpy()

    # Handle grayscale mask (2D)
    if is_mask or t.ndim == 2:
        t = (t * 255).clip(0, 255).astype(np.uint8)
        return Image.fromarray(t, mode='L')

    # Handle RGB image (3D)
    if t.ndim == 3:
        # If channels first (C, H, W), convert to (H, W, C)
        if t.shape[0] == 3 or t.shape[0] == 1:
            t = np.transpose(t, (1, 2, 0))
        t = (t * 255).clip(0, 255).astype(np.uint8)
        return Image.fromarray(t)

    raise ValueError(f"Unexpected tensor shape: {t.shape}")

print(f"Evaluating on {NUM_SAMPLES} samples...")

for _ in tqdm(range(NUM_SAMPLES)):
    idx = random.randint(0, len(dataset) - 1)
    sample = dataset[idx]

    # Convert tensors to PIL images
    clean_pil = tensor_to_pil(sample["clean"], is_mask=False)
    damaged_pil = tensor_to_pil(sample["damaged"], is_mask=False)
    mask_pil = tensor_to_pil(sample["mask"], is_mask=True)

    # Restore image
    with torch.no_grad():
        restored = pipe(
            prompt="professional photo restoration, repair damage completely, preserve identity and details, natural appearance",
            negative_prompt="blur, artifacts, scratches, cracks, distortion, fake",
            image=damaged_pil,
            mask_image=mask_pil,
            num_inference_steps=60,
            guidance_scale=7.0,
            strength=0.90
        ).images[0]

    # Compute metrics
    clean_np = np.array(clean_pil).astype(np.float32)
    damaged_np = np.array(damaged_pil).astype(np.float32)
    restored_np = np.array(restored).astype(np.float32)

    # Calculate PSNR and SSIM
    p_d = psnr(clean_np, damaged_np, data_range=255)
    s_d = ssim(clean_np, damaged_np, data_range=255, channel_axis=2)
    p_r = psnr(clean_np, restored_np, data_range=255)
    s_r = ssim(clean_np, restored_np, data_range=255, channel_axis=2)

    psnr_damaged_list.append(p_d)
    ssim_damaged_list.append(s_d)
    psnr_restored_list.append(p_r)
    ssim_restored_list.append(s_r)

# Final results
print("\n" + "="*60)
print("FINAL EVALUATION RESULTS")
print("="*60)

print(f"\nDamaged  → Clean | PSNR: {np.mean(psnr_damaged_list):.2f} ± {np.std(psnr_damaged_list):.2f} dB | SSIM: {np.mean(ssim_damaged_list):.4f} ± {np.std(ssim_damaged_list):.4f}")
print(f"Restored → Clean | PSNR: {np.mean(psnr_restored_list):.2f} ± {np.std(psnr_restored_list):.2f} dB | SSIM: {np.mean(ssim_restored_list):.4f} ± {np.std(ssim_restored_list):.4f}")

print(f"\nIMPROVEMENT:")
psnr_gain = np.mean(psnr_restored_list) - np.mean(psnr_damaged_list)
ssim_gain = np.mean(ssim_restored_list) - np.mean(ssim_damaged_list)

print(f"PSNR Gain: {psnr_gain:+.2f} dB ({'+' if psnr_gain > 0 else ''}{(psnr_gain/np.mean(psnr_damaged_list))*100:.1f}%)")
print(f"SSIM Gain: {ssim_gain:+.4f} ({'+' if ssim_gain > 0 else ''}{(ssim_gain/np.mean(ssim_damaged_list))*100:.1f}%)")
print("="*60)

# Show best and worst restorations
best_idx = np.argmax(np.array(psnr_restored_list) - np.array(psnr_damaged_list))
worst_idx = np.argmin(np.array(psnr_restored_list) - np.array(psnr_damaged_list))

print(f"\nBest restoration improvement: +{psnr_restored_list[best_idx] - psnr_damaged_list[best_idx]:.2f} dB")
print(f"Worst restoration improvement: {psnr_restored_list[worst_idx] - psnr_damaged_list[worst_idx]:+.2f} dB")

In [ ]:
# Create output directory for restored images
RESTORED_DIR = Path("/content/drive/MyDrive/DL_Project/restored images")
RESTORED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Restoring and saving all damaged images to: {RESTORED_DIR}")
print("="*60)

# Process all damaged images
damaged_files = sorted([f for f in os.listdir(DAMAGED_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

for fname in tqdm(damaged_files, desc="Restoring images"):
    try:
        # Load images
        damaged_pil = Image.open(DAMAGED_DIR / fname).convert("RGB").resize((512, 512))
        mask_pil = Image.open(MASK_DIR / fname).convert("L").resize((512, 512))

        # Perform restoration
        with torch.no_grad():
            restored_pil = pipe(
                prompt="professional museum-quality photo restoration, repair all damage, preserve original details and colors, natural skin texture, sharp details",
                negative_prompt="blur, artifacts, distortion, oversaturated, fake, synthetic, unnatural, scratches, cracks",
                image=damaged_pil,
                mask_image=mask_pil,
                num_inference_steps=75,
                guidance_scale=7.5,
                strength=0.95
            ).images[0]

        # Save restored image
        output_name = Path(fname).stem + "_restored.png"
        restored_pil.save(RESTORED_DIR / output_name, format="PNG")

    except Exception as e:
        print(f"Error restoring {fname}: {e}")

print(f"\nAll restored images saved to: {RESTORED_DIR}")
print(f"Total images restored: {len(list(RESTORED_DIR.glob('*.png')))}")